# 简介

[Pydantic](https://docs.pydantic.dev/latest/)用于模式验证,类似`npm`的`zod`库.


# 示例

## 基本使用

In [4]:
import json
from datetime import datetime

from pydantic import BaseModel, PositiveInt


class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data = {
    'id': 123,
    'signup_ts': '2019-06-01 12:22',
    'tastes': {
        'wine': 9,
        b'cheese': 7,
        'cabbage': '1',
    },
}

user = User(**external_data)

print(user.id)
#> 123
print(user.model_dump())
"""
{
    'id': 123,
    'name': 'John Doe',
    'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
    'tastes': {'wine': 9, 'cheese': 7, 'cabbage': 1},
}
"""

ValidationError: 1 validation error for User
tastes.cabbage
  Input should be greater than 0 [type=greater_than, input_value='-1', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/greater_than

## 验证失败

验证失败时会抛出`ValidationError`异常,可以通过`e.errors()`获取错误信息.

In [8]:
# continuing the above example...

from pydantic import ValidationError
import json

class User(BaseModel):
    id: int
    name: str = 'John Doe'
    signup_ts: datetime | None
    tastes: dict[str, PositiveInt]


external_data = {'id': 'not an int', 'tastes': {}}

try:
    User(**external_data)
except ValidationError as e:
    print(json.dumps(e.errors(), indent=4))
    """
    [
        {
            'type': 'int_parsing',
            'loc': ('id',),
            'msg': 'Input should be a valid integer, unable to parse string as an integer',
            'input': 'not an int',
            'url': 'https://errors.pydantic.dev/2/v/int_parsing',
        },
        {
            'type': 'missing',
            'loc': ('signup_ts',),
            'msg': 'Field required',
            'input': {'id': 'not an int', 'tastes': {}},
            'url': 'https://errors.pydantic.dev/2/v/missing',
        },
    ]
    """

[
    {
        "type": "int_parsing",
        "loc": [
            "id"
        ],
        "msg": "Input should be a valid integer, unable to parse string as an integer",
        "input": "not an int",
        "url": "https://errors.pydantic.dev/2.8/v/int_parsing"
    },
    {
        "type": "missing",
        "loc": [
            "signup_ts"
        ],
        "msg": "Field required",
        "input": {
            "id": "not an int",
            "tastes": {}
        },
        "url": "https://errors.pydantic.dev/2.8/v/missing"
    }
]
